* grouplens MovieLens 2018 20M data: https://grouplens.org/datasets/movielens/20m/
* Readme: http://files.grouplens.org/datasets/movielens/ml-20m-README.html


In [1]:
# import dependencies
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

In [2]:
conn = sqlite3.connect('2018Movies20M.sqlite')

In [3]:
c = conn.cursor()
conn.execute('drop table if exists movie_ratings')

In [11]:
c.execute('''CREATE TABLE movie_ratings
    (id INT PRIMARY KEY,
    userId INT,
    movieId INT,
    rating INT,
    no_genre INT,
    Action INT,
    Adventure INT, 
    Animation INT, 
    Children INT, 
    Comedy INT, 
    Crime INT,
    Documentary INT, 
    Drama INT, 
    Fantasy INT, 
    Film_Noir INT, 
    Horror INT, 
    IMAX INT,
    Musical INT, 
    Mystery INT, 
    Romance INT, 
    Sci_Fi INT, 
    Thriller INT, 
    War INT,
    Western INT)''')

In [12]:
conn.commit()

In [13]:
# importing the ratings file
ratings = pd.read_csv('ml-20m/ratings.csv', encoding='latin-1')
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [14]:
# importing movies file
items = pd.read_csv('ml-20m/movies.csv', encoding='latin-1')
items.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
# merge users and ratings
df = pd.merge(ratings, items, on="movieId")
df.sort_values("userId").head()

,userId,movieId,rating,timestamp,title,genres
0,1,2,3.5,1112486027,Jumanji (1995),Adventure|Children|Fantasy
505014,1,541,4.0,1112484603,Blade Runner (1982),Action|Sci-Fi|Thriller
2380423,1,6807,3.5,1112484686,Monty Python's The Meaning of Life (1983),Comedy
2378699,1,6774,4.0,1112485786,Videodrome (1983),Fantasy|Horror|Sci-Fi|Thriller
2376750,1,6755,3.5,1094786067,Bubba Ho-tep (2002),Comedy|Horror


In [16]:
# check for null values
df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
title        0
genres       0
dtype: int64

In [17]:
# used one hot encoding to transform genres into features
genres = pd.get_dummies(df.genres.str.split('|',expand=True).stack()).sum(level=0)
genres.head()

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [18]:
# merge genres back to the original df
df = pd.concat([df,genres], axis=1)
df.head()

,userId,movieId,rating,timestamp,title,genres,(no genres listed),Action,Adventure,Animation,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,2,3.5,1112486027,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,5,2,3.0,851527569,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,13,2,3.0,849082742,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,29,2,3.0,835562174,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,34,2,3.0,846509384,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# use drop to get remaining features
data = df.drop(['timestamp', 'title', 'genres'],axis=1, inplace=True)

In [20]:
df.head()

,userId,movieId,rating,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,2,3.5,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,2,3.0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,13,2,3.0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,29,2,3.0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,34,2,3.0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
# datacsv = df.to_csv("../201820M.csv")
df.columns.values

array(['userId', 'movieId', 'rating', '(no genres listed)', 'Action',
       'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'], dtype=object)

In [22]:
df = df.rename(index=str, columns={"(no genres listed)":"no_genre", "Film-Noir":"Film_Noir", "Sci-Fi":"Sci_Fi"})

In [23]:
df.to_sql('movie_ratings', conn, if_exists='append', index=True, index_label='id')
